In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Web Mining/content

/content/drive/MyDrive/Web Mining/content


In [3]:
try : 
  import scrapy
except : 
  !pip install scrapy
  import scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 13.3 MB/s 
     |████████████████████████████████| 4.1 MB 61.4 MB/s 
     |████████████████████████████████| 251 kB 60.9 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 3.1 MB 45.4 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=057ef1467163f7fa1135174cffbb1ef15c5d6d0cb0f50a96a203b79782eb9d68
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


In [4]:
import pandas as pd

In [ ]:
class LinkSpider(scrapy.Spider):
    name = 'link'
    start_urls=[]
    for i in range (1, 50+1):
        start_urls.append(f'https://pta.trunojoyo.ac.id/c_search/byprod/10/{i}')
    def parse(self, response):
        count = 0 
        link = []
        for jurnal in response.css('#content_journal > ul'):
            count += 1
            for j in range(1, 6):
                yield {
                    'link' : response.css(f'li:nth-child({j}) > div:nth-child(3) > a::attr(href)').get(),
                }


In [8]:
%cd /content/drive/MyDrive/Web Mining/webmining

/content/drive/MyDrive/Web Mining/webmining


In [13]:
linkHasilCrawl = pd.read_csv('hasilCrawlLink.csv')
linkHasilCrawl

,link
0,https://pta.trunojoyo.ac.id/welcome/detail/040...
1,https://pta.trunojoyo.ac.id/welcome/detail/040...
2,https://pta.trunojoyo.ac.id/welcome/detail/040...
3,https://pta.trunojoyo.ac.id/welcome/detail/070...
4,https://pta.trunojoyo.ac.id/welcome/detail/080...
...,...
95,https://pta.trunojoyo.ac.id/welcome/detail/090...
96,https://pta.trunojoyo.ac.id/welcome/detail/080...
97,https://pta.trunojoyo.ac.id/welcome/detail/080...
98,https://pta.trunojoyo.ac.id/welcome/detail/090...


In [ ]:
class Spider(scrapy.Spider):
    name = 'detail'
    data_link = pd.read_json('hasilLink.json').values
    start_urls = [link[0] for link in data_link]

    def parse(self, response):
        yield{
            'abstraksi' : response.css('#content_journal > ul > li > div:nth-child(4) > div:nth-child(2) > p::text').extract(),
        }

In [ ]:
df = pd.read_csv('hasilCrawl.csv')

In [ ]:
!pip install nltk
!pip install Sastrawi

In [ ]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
data = pd.read_csv('hasilCrawl.csv')


In [ ]:
def remove_stopwords(text):
    with open('stopword.txt') as f:
        stopwords = f.readlines()
        stopwords = [x.strip() for x in stopwords]

    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]

    return text

In [ ]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    result = [stemmer.stem(word) for word in text]

    return text

In [ ]:
def preprocessing(text):
    #case folding
    text = text.lower()
    
    #remove urls
    text = re.sub('http\S+', '', text)
    
    #replace weird characters
    text = text.replace('“', '"')
    text = text.replace('”', '"')
    text = text.replace('-', ' ')
            
    #tokenization and remove stopwords
    text = remove_stopwords(text)
    
    #remove punctuation    
    text = [''.join(c for c in s if c not in string.punctuation) for s in text]    
    
    #stemming
    text = stemming(text)
    
    #remove empty string
    text = list(filter(None, text))
    
    return text

In [ ]:
tf = pd.DataFrame()
for i,v in enumerate(data['Abstraksi']):
    cols = ["Doc " + str(i+1)]    
    doc = pd.DataFrame.from_dict(nltk.FreqDist(preprocessing(v)), orient='index',columns=cols) 
    #doc.columns = [data['Judul'][i]]    
    tf = pd.concat([tf, doc], axis=1, sort=False)

In [ ]:
tf.index.name = 'Term'
tf = pd.concat([tf], axis=1, sort=False)
tf = tf.fillna(0)
tf